Imports et Configuration

In [1]:
import spacy
import requests
import xml.etree.ElementTree as ET
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.metrics.distance import edit_distance
import numpy as np


Pré-traitement des Questions

In [2]:
# Charger le modèle NLP pour le français (ou anglais selon les questions utilisées)
nlp = spacy.load("en_core_web_sm")  # Change en 'fr_core_news_sm' si questions en français


Recherche d'Entités DBpedia

In [3]:
def preprocess_question(question):
    print(f"\nTraitement de la question: {question}")
    tokens = word_tokenize(question)
    tokens = [t.lower() for t in tokens if t.isalnum() and t not in stopwords.words('english')]
    doc = nlp(question)
    named_entities = [(ent.text, ent.label_) for ent in doc.ents]
    print("Tokens:", tokens)
    print("Entités Nommées:", named_entities)
    return tokens, named_entities


Chargement du Fichier XML

In [4]:
def parse_questions_xml(file_path):
    print("\nLecture du fichier questions.xml...")
    tree = ET.parse(file_path)
    root = tree.getroot()
    questions = []

    for question in root.findall('question'):
        question_text = question.find("string[@lang='en']")
        answers_element = question.find('answers')

        if question_text is not None:
            question_text = question_text.text
        else:
            print("⚠️ Question sans texte trouvé, passage...")
            continue  # Ignore la question si aucun texte n'est trouvé

        if answers_element is not None:
            expected_answer = [ans.find('uri').text for ans in answers_element.findall('answer') if ans.find('uri') is not None]
        else:
            print(f"⚠️ Pas de réponse trouvée pour la question: {question_text}")
            expected_answer = []

        questions.append((question_text, expected_answer))

    print(f"Nombre de questions chargées: {len(questions)}")
    return questions


Traitement des Questions

In [5]:
questions_file = "questions.xml"
questions_data = parse_questions_xml(questions_file)



Lecture du fichier questions.xml...
Nombre de questions chargées: 25


Création de la Requête SPARQL

In [6]:
def process_question(question_text, expected_answer):
    print(f"### Question ###\n")
    print(f"Traitement de la question: {question_text}")
    
    tokens, named_entities = preprocess_question(question_text)
    print(f"Tokens: {tokens}")
    print(f"Entités Nommées: {named_entities}")
    
    print(f"Réponse attendue: {expected_answer}")
    
    # Essayer de trouver une entité DBpedia
    for entity, label in named_entities:
        uri = find_dbpedia_entity(entity)
        print(f"Entité DBpedia trouvée pour {entity}: {uri}")


In [7]:
# Appeler la fonction pour afficher les détails de la première question
process_question("Which river does the Brooklyn Bridge cross?", ['http://dbpedia.org/resource/East_River'])
# Appeler la fonction pour afficher les détails de la deuxième question
process_question("Who is the author of Wikipedia?", ['http://dbpedia.org/resource/Jimmy_Wales', 'http://dbpedia.org/resource/Larry_Sanger'])


### Question ###

Traitement de la question: Which river does the Brooklyn Bridge cross?

Traitement de la question: Which river does the Brooklyn Bridge cross?
Tokens: ['which', 'river', 'brooklyn', 'bridge', 'cross']
Entités Nommées: [('the Brooklyn Bridge', 'ORG')]
Tokens: ['which', 'river', 'brooklyn', 'bridge', 'cross']
Entités Nommées: [('the Brooklyn Bridge', 'ORG')]
Réponse attendue: ['http://dbpedia.org/resource/East_River']


NameError: name 'find_dbpedia_entity' is not defined

In [74]:
# Appeler la fonction pour afficher les détails de la troisième question
process_question("In which country does the Nile start?", ['http://dbpedia.org/resource/Ethiopia', 'http://dbpedia.org/resource/Lake_Tana'])
# Appeler la fonction pour afficher les détails de la quatrième question
process_question("What is the highest place of Karakoram?", ['http://dbpedia.org/resource/K2'])

### Question ###

Traitement de la question: In which country does the Nile start?

Traitement de la question: In which country does the Nile start?
Tokens: ['in', 'country', 'nile', 'start']
Entités Nommées: [('Nile', 'LOC')]
Tokens: ['in', 'country', 'nile', 'start']
Entités Nommées: [('Nile', 'LOC')]
Réponse attendue: ['http://dbpedia.org/resource/Ethiopia', 'http://dbpedia.org/resource/Lake_Tana']

Recherche de l'entité DBpedia pour: Nile
Entité DBpedia trouvée: ['http://dbpedia.org/resource/Nile_Rodgers']
Entité DBpedia trouvée pour Nile: ['http://dbpedia.org/resource/Nile_Rodgers']
### Question ###

Traitement de la question: What is the highest place of Karakoram?

Traitement de la question: What is the highest place of Karakoram?
Tokens: ['what', 'highest', 'place', 'karakoram']
Entités Nommées: [('Karakoram', 'GPE')]
Tokens: ['what', 'highest', 'place', 'karakoram']
Entités Nommées: [('Karakoram', 'GPE')]
Réponse attendue: ['http://dbpedia.org/resource/K2']

Recherche de l'ent

In [75]:
# Appeler la fonction pour afficher les détails de la cinquième question
process_question("Who designed the Brooklyn Bridge?", ['http://dbpedia.org/resource/John_Augustus_Roebling'])

### Question ###

Traitement de la question: Who designed the Brooklyn Bridge?

Traitement de la question: Who designed the Brooklyn Bridge?
Tokens: ['who', 'designed', 'brooklyn', 'bridge']
Entités Nommées: [('the Brooklyn Bridge', 'FAC')]
Tokens: ['who', 'designed', 'brooklyn', 'bridge']
Entités Nommées: [('the Brooklyn Bridge', 'FAC')]
Réponse attendue: ['http://dbpedia.org/resource/John_Augustus_Roebling']

Recherche de l'entité DBpedia pour: the Brooklyn Bridge
Entité DBpedia trouvée: ['http://dbpedia.org/resource/The_Brooklyn_Bridge_(band)']
Entité DBpedia trouvée pour the Brooklyn Bridge: ['http://dbpedia.org/resource/The_Brooklyn_Bridge_(band)']


Évaluation du Système

In [76]:
def create_sparql_query(subject, relation):
    print(f"\nCréation de la requête SPARQL pour le sujet: {subject} et la relation: {relation}")
    query = f"""
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX res: <http://dbpedia.org/resource/>
    SELECT DISTINCT ?uri WHERE {{
        res:{subject} dbo:{relation} ?uri .
    }}
    """
    print("Requête SPARQL générée:")
    print(query)
    return query


Chargement des Données et Exécution

In [77]:
sparql_query = create_sparql_query("Brooklyn_Bridge", "crosses")



Création de la requête SPARQL pour le sujet: Brooklyn_Bridge et la relation: crosses
Requête SPARQL générée:

    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX res: <http://dbpedia.org/resource/>
    SELECT DISTINCT ?uri WHERE {
        res:Brooklyn_Bridge dbo:crosses ?uri .
    }
    


Analyse des Erreurs

### Analyse des Erreurs

Sur quel type de question votre système se trompe ?

1. **Questions ambiguës** : Si une question contient plusieurs entités ou relations possibles, le système peut avoir du mal à choisir la bonne.
2. **Mauvais matching des relations** : Certaines relations peuvent ne pas être correctement identifiées en raison de variations lexicales ou de synonymes.
3. **Absence d'entités dans DBpedia** : Si une entité mentionnée dans la question n'existe pas dans DBpedia, le système ne pourra pas trouver de correspondance.

### Améliorations Possibles

- Utiliser des techniques avancées pour le matching des relations, comme la similarité sémantique avec WordNet.
- Ajouter un mécanisme de résolution des ambiguïtés en proposant plusieurs candidats pour chaque entité ou relation.
- Tester sur un ensemble plus large de questions pour identifier les cas limites.

In [78]:
def evaluate_system(predictions, gold_standard):
    print("\nÉvaluation du système")
    tp = sum(1 for p in predictions if p in gold_standard)
    fp = sum(1 for p in predictions if p not in gold_standard)
    fn = sum(1 for g in gold_standard if g not in predictions)
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    f_measure = (2 * precision * recall) / (precision + recall) if precision + recall > 0 else 0
    print(f"Précision: {precision}, Rappel: {recall}, F-mesure: {f_measure}")
    return precision, recall, f_measure


In [79]:
predicted_answers = ["http://dbpedia.org/resource/East_River"]
gold_answers = ["http://dbpedia.org/resource/East_River"]
evaluate_system(predicted_answers, gold_answers)



Évaluation du système
Précision: 1.0, Rappel: 1.0, F-mesure: 1.0


(1.0, 1.0, 1.0)